# Revised Class Definitions

These are classes to replace and enhance the prior versions of the nest handlers, refactored for better isolation.
![Structure](../../media/R30/R30-b-overall-structure.png)

In [ ]:
// |> start Docker database <(-- only once!
#!powershell

docker start awsql

In [ ]:
// |> Import Nuget packages
#r "nuget: Microsoft.Data.SqlClient"

In [ ]:
// |> import the existing library code 
#!import "../code/composite.cs"

In [ ]:
// |> import credentials
#!import ..\.data\connect-Aw-credentials.csx.cs
Console.WriteLine("You can now use the \"_credentials\" string for the JsonSqlClient constructor");

In [ ]:
// |> MAIN CLASSES UNDER DEVELOPMENT //
#nullable enable
public class SqlSchemaModeler
{
	public string Diagnostic { get; /*private*/ set; } = "";
	internal Dictionary<string, int> _lookup = new Dictionary<string, int>();
	private const string _query_system_relationships_ =
		 " with Data as "
	   + " (select "
	   + " table_schema SchemaName, "
	   + " table_name TableName, "
	   + " column_name FieldName, "
	   + " lower(table_schema + '.' + table_name + '.' + column_name) as 'identifier', "
	   + " CASE is_nullable WHEN 'yes' Then '1' ELSE '0' END as IsNullable, "
	   + " DATA_TYPE as FieldDataType from information_schema.columns "
	   + " ), "
	   + " fk as "
	   + " ( select "
	   + " lower(kcu2.table_schema + '.' + kcu2.table_name + '.' + kcu2.column_name) as Identifier, "
	   + " lower(kcu1.table_schema + '.' + kcu1.table_name + '.' + kcu1.column_name) as Reference "
	   + " from information_schema.referential_constraints rc "
	   + " join information_schema.key_column_usage kcu1 on kcu1.constraint_catalog = rc.constraint_catalog "
	   + " and kcu1.constraint_schema = rc.constraint_schema "
	   + " and kcu1.constraint_name = rc.constraint_name "
	   + " join information_schema.key_column_usage kcu2 on kcu2.constraint_catalog = rc.unique_constraint_catalog "
	   + " and kcu2.constraint_schema = rc.unique_constraint_schema "
	   + " and kcu2.constraint_name = rc.unique_constraint_name "
	   + " and kcu2.ordinal_position = kcu1.ordinal_position "
	   + " ), "
	   + " pk as "
	   + " ( "
	   + " select "
	   + " lower(table_schema + '.' + table_name + '.' + column_name) as Identifier, "
	   + " '1' as IsPrimary "
	   + " from information_schema.key_column_usage "
	   + " where CONSTRAINT_NAME like 'PK_%' "
	   + " ) "
	   + " select  data.Identifier, SchemaName, TableName, FieldName , FieldDataType, isnull(Reference, '') Reference, isnull(IsPrimary,  '0') IsPrimary, IsNullable "
	   + " from data left join fk on data.[identifier] = fk.identifier "
	   + " left join pk on data.[identifier] =  pk.Identifier "
	   + " -- where -- "
	   + " for json path, include_null_values ";

	private JsonSqlClient _client;
	public int FieldCount => Model == null ? 0 : Model.SchemaFields.Length;
	public SqlSchemaModeler(string _connectionString)
	{
		_client = new JsonSqlClient(_connectionString);
	}
	public async Task<string[]> GetSchemaNames()
	{
		await Task.CompletedTask;
		return _client.GetSchemaNames();
	}
	public async Task InitializeAsync(params string[] schemas)
	{
		Model = new SchemaModel();
		Model.SchemaFields = await GetSchemaFields(schemas);

		await RationalizeModel();
	}
	private async Task<SchemaField[]> GetSchemaFields(string[] schemas)
	{
		string sql = _query_system_relationships_;
		await Task.CompletedTask;

		if (schemas.Length > 0)
		{
			string x = String.Join(',', schemas).Replace(",", "','").Replace("[", "").Replace("]", "");
			sql = sql.Replace("-- where --", $" where SchemaName in ( '{x}' ) ");
		}
		var r = _client.JsonSingleUsingQuery(sql);
		Diagnostic = r;
		var sf = JsonSerializer.Deserialize<SchemaField[]>(r);

		return sf.DistinctBy(sf => sf.Identifier).ToArray();
	}
	public SchemaField this[int index]
	{
		get
		{
			if (Model.SchemaFields.Length > index && index > -1)
				return Model.SchemaFields[index];
			return new SchemaField();
		}
	}
	public string ToJson()
	{
		return Model.ToJson();
	}
	public void FromJson(string modelJson)
	{
		Model = JsonSerializer.Deserialize<SchemaModel>(modelJson);
		RationalizeModel();
	}
	public SchemaField this[string identifier]
	{
		get
		{
			if (_lookup.ContainsKey(identifier))
				return Model.SchemaFields[_lookup[identifier]];
			return new SchemaField();
		}
	}
	private async Task RationalizeModel()
	{
		// update the model calculables
		int ix = 0;
		_lookup = new Dictionary<string, int>();
		// make the identifier-index lookup
		foreach (var f in Model.SchemaFields)
		{
			Diagnostic += $"{f.Index:0##}={f.Identifier}\n";
			f.Index = ix++;
			_lookup.Add(f.Identifier, f.Index);
		}
		// if there is a reference
		//    if in dictionary, internal
		//    otherwise external
		// if not then NotAReference.
		foreach (var f in Model.SchemaFields.Where(f => !string.IsNullOrEmpty(f.Reference)))
		{
			if (_lookup.ContainsKey(f.Reference))
			{
				f.ReferenceType = FieldReferenceType.Internal;
			}
			else
			{
				f.ReferenceType = FieldReferenceType.External;
			}
		}
		await Task.CompletedTask;
	}
	public async Task<int> InferReferences(bool includeShortform = true)
	{
		int count = 0;
		await Task.CompletedTask;
		// Make a list of all PKs that match the convention.
		Dictionary<string, string> tempLookup = new Dictionary<string, string>();
		foreach (var f in Model.SchemaFields.Where(f => f.IsPrimary == "1").DistinctBy(f => f.Identifier))
		{
			// check that this is matches the convention
			if (f.FieldName.ToLower() == $"{f.TableName.ToLower()}id")
			{
				tempLookup.Add(f.FieldName.ToLower(), f.Identifier);
			}
		}
		// If the normal convention of naming the PK with the table name + Id, we may infer those
		foreach (var f in Model.SchemaFields.Where(f => f.IsPrimary != "1" && f.ReferenceType == FieldReferenceType.NotAReference))
		{
			if (tempLookup.ContainsKey(f.FieldName.ToLower()))
			{
				f.Reference = tempLookup[f.FieldName.ToLower()];
				f.ReferenceType = FieldReferenceType.Inferred;
				count++;
			}
		}
		// Alternative Short Form convention where PK is always ID, but is prefixed with the singularized table name when referenced
		// e.g. table = MyElements pk = Id, fk = MyElementId
		foreach (var f in Model.SchemaFields.Where(f => f.IsPrimary == "1" && f.FieldName.ToLower() == "id"))
		{
			var correctedKey = $"{f.TableName.ToLower()}id";
			var correctedReference = $"{f.SchemaName}.{f.TableName.ToLower()}.id";
			foreach (var g in Model.SchemaFields.Where(f => f.IsPrimary != "1" && f.ReferenceType == FieldReferenceType.NotAReference))
			{
				if (tempLookup.ContainsKey(correctedKey))
				{
					g.Reference = correctedReference;
					g.ReferenceType = FieldReferenceType.Inferred;
					count++;
				}
			}
		}
		return count;
	}
	[Flags]
	public enum Navigability
	{
	    None = 0,
		InBound = 1,
		OutBound = 2,
		Both = 3
	}
	/////////////////////////////////////////////////////////////////////////////////////////////////////
	////
	public List<string> IdentifyReferences(string initialIdentifier, int maxDepth = 8, Navigability navigability = Navigability.Both, List<string>? result = null)
	{
		maxDepth--;
		if (result == null) result = new List<string>();
		if (maxDepth == 0)
		{
			return result;
		}
		// Inbound from FKs to this PK
		if ((navigability & Navigability.InBound) == Navigability.InBound)
		{
			var target =  Model.SchemaFields.Where(f => f.Reference == initialIdentifier).DistinctBy(f => f.Identifier).Select(f => f.Identifier);
			foreach (var e in target)
			{
				if (!WasTraversed(e))
				{
					result.Add(e);
					var x = IdentifyReferences(e, maxDepth + 1, navigability, result).Where(e => !WasTraversed(e));
					result.AddRange(x);
				}
			}
			
		}
		// Outbound
		if ((navigability & Navigability.OutBound) == Navigability.OutBound)
		{
			var root = Model.SchemaFields.Where(f => f.Identifier == initialIdentifier).First();
			if (!string.IsNullOrEmpty(root.Reference) && !WasTraversed(root.Reference))
			   result.Add(root.Reference);
			var target = Model.SchemaFields
				.Where(f => f.SchemaName == root.SchemaName && f.TableName == root.TableName && !string.IsNullOrEmpty(f.Reference))
				.DistinctBy(f => f.Identifier)
				.Select(f => f.Identifier);
			foreach (var e in target)
			{
				if (!WasTraversed(e))
				{
					result.Add(e);
					var x = IdentifyReferences(e, maxDepth, navigability, result).Where(e => !WasTraversed(e));
					result.AddRange(x);
				}
			}
		}
		bool WasTraversed(string identifier)
		{
			return result.Contains(identifier);
		}
		return result;
	}
	////
	/////////////////////////////////////////////////////////////////////////////////////////////////////


	public async Task Refresh(params string[] schemas)
	{
		var oldFields = Model.SchemaFields;
		await InitializeAsync(schemas);
		await Overlay(oldFields);
		await RationalizeModel();
	}
	public async Task Overlay(params SchemaField[] updates)
	{
		foreach (var inbound in updates)
		{
			if (_lookup.ContainsKey(inbound.Identifier))
			{
				int index = _lookup[inbound.Identifier];
				var existing = Model.SchemaFields[index];
				if (!String.IsNullOrEmpty(inbound.Reference))
					existing.Reference = inbound.Reference;
				if (inbound.IsDisabled.HasValue)
					existing.IsDisabled = inbound.IsDisabled;
				if (inbound.IsExcluded.HasValue)
					existing.IsExcluded = inbound.IsExcluded;
				if (!string.IsNullOrEmpty(inbound.Intervention))
					existing.Intervention = inbound.Intervention;
			}
			else
			{
				var temp = Model.SchemaFields.ToList();
				temp.Add(inbound);
				Model.SchemaFields = temp.ToArray();
			}
		}
		await RationalizeModel();
	}
	public async Task<int> ExcludebyPath(string twoDotPath) => await ExcludeByPath(twoDotPath, true);
	public async Task<int> IncludebyPath(string twoDotPath) => await ExcludeByPath(twoDotPath, false);
	// return the number of fields matched.
	private async Task<int> ExcludeByPath(string twoDotPath, bool yes)
	{
		await Task.CompletedTask;
		IEnumerable<SchemaField> selection = new SchemaField[0]; var selector = twoDotPath.ToLower().Split('.', StringSplitOptions.None);
		if (selector.Length == 1)
		{  // just select based on Schema
			selection = Model.SchemaFields.Where(f =>
				Matches(f.SchemaName, selector[0]));
		}
		else if (selector.Length == 2)
		{
			selection = Model.SchemaFields.Where(f =>
			   Matches(f.SchemaName, selector[0]) &&
			   Matches(f.TableName, selector[1]));
		}
		else if (selector.Length == 3)
		{
			selection = Model.SchemaFields.Where(f =>
				Matches(f.SchemaName, selector[0]) &&
				Matches(f.TableName, selector[1]) &&
				Matches(f.FieldName, selector[2]));
		}
		foreach (var f in selection)
		{
			f.IsExcluded = yes;
		}
		return selection.Count();
		bool Matches(string evidence, string pattern)
		{
			evidence = evidence.ToLower();
			pattern = pattern.ToLower();
			if (pattern.Length == 0 || pattern == "*")
				return true;
			if (pattern.Length > 1 && pattern.StartsWith("*"))
				return (evidence.EndsWith(pattern.Substring(1)));
			if (pattern.Length > 1 && pattern.EndsWith("*"))
				return (evidence.StartsWith(pattern.Substring(0, pattern.Length - 1)));
			else
				return evidence == pattern;
		}
	}
	public async Task Prune(bool disabled, bool excluded)
	{
		var temp = Model.SchemaFields.Where(f =>
		   !(((f.IsDisabled.HasValue && f.IsDisabled.Value) && disabled) ||
		   ((f.IsExcluded.HasValue && f.IsExcluded.Value) && excluded))
		);
		Diagnostic = $"{temp.Count()}";
		Model.SchemaFields = temp.ToArray();
		await RationalizeModel();
	}
	public SchemaModel Model { get; set; } = new SchemaModel();
}

// public class SqlModelQuery
// {
// 	private JsonSqlClient _client;
// 	public SqlModelQuery(string connectionString, SchemaModel model)
// 	{
// 		_client = new JsonSqlClient(connectionString);
// 	}
// 	public async Task<ModelFilter[]> GetFilters()
// 	{
// 		await Task.CompletedTask;
// 		return new ModelFilter[0];
// 	}
// 	public async Task Execute(params ModelFilter[] filters)
// 	{
// 		await Task.CompletedTask;
// 	}
// }
public class NestedDataModeler
{
	private SchemaModel _model;
	private string _queryResult;
	public NestedDataModeler(SchemaModel model, string queryResult)
	{
		_model = model;
		_queryResult = queryResult;
	}
	public async Task<NestedData> Execute()
	{
		await Task.CompletedTask;
		return new NestedData();
	}
}
public class NestedDataPresenter
{
	private PresenterOptions _options = new PresenterOptions();
	public NestedDataPresenter()
	{

	}
	public async Task<string[]> GetPages(PageType pageType, params NestedData[] sources)
	{
		await Task.CompletedTask;
		return new string[0];
	}
}
[Flags]
public enum PageType
{
	Html = 1,
	Css = 2,
	Script = 4,
	Scrolled = 8,
	Stepped = 16,
	Vast = 32
}
public enum FieldReferenceType
{
	NotAReference,
	Internal, // reference is contained within this model
	External, // reference is outside of this model
	Inferred  // name matches a local id
}
public class SchemaField : Edge
{
	// base fields
	public string SchemaName { get; set; } = "";
	public string TableName { get; set; } = "";
	public string FieldName { get; set; } = "";
	public FieldReferenceType ReferenceType { get; set; } = FieldReferenceType.NotAReference;
	public string FieldDataType { get; set; } = "unknown";
	public string IsNullable { get; set; } = "0";
	public string IsPrimary { get; set; } = "0";
	// overlay fields
	public int Index { get; set; } = -1;
	public bool? IsDisabled { get; set; }
	public bool? IsExcluded { get; set; }
	public string? Intervention { get; set; }
	public bool CanShow => !(IsDisabled == true || IsExcluded == true);
	// diagnostic display
	public override string ToString()
	{
		string r = (string.IsNullOrEmpty(Reference) ? "" : $"{ReferenceType} Reference {Reference}");
		return $"{Index} {Identifier} ({ReferenceType}) {r}";
	}
	// added to overcome scope issues while testing in a notebook
	public bool IsInferredReference => ReferenceType == FieldReferenceType.Inferred;
	public bool IsInternalReference => ReferenceType == FieldReferenceType.Internal;
	public bool IsExternalRefernce => ReferenceType == FieldReferenceType.External;
	public bool IsNotAReference => ReferenceType == FieldReferenceType.NotAReference;

}
public class Edge
{
	public string Identifier { get; set; } = "";
	// base if FK-PK else overlay if manual, inferred
	public string? Reference { get; set; }
	public override string ToString()
	{
		return $"{Identifier}!{Reference}";
	}
}
public class SchemaModel
{
	public SchemaField[] SchemaFields { get; set; } = new SchemaField[0];
	public string ToJson()
	{
		return System.Text.Json.JsonSerializer.Serialize(this);
	}
	public async Task<int> InferReferences(bool includeShortform = true)
	{
		int count = 0;
		await Task.CompletedTask;
		// Make a list of all PKs that match the convention.
		Dictionary<string, string> tempLookup = new Dictionary<string, string>();
		foreach (var f in SchemaFields.Where(f => f.IsPrimary == "1").DistinctBy(f => f.Identifier))
		{
			// check that this is matches the convention
			if (f.FieldName.ToLower() == $"{f.TableName.ToLower()}id")
			{
				tempLookup.Add(f.FieldName.ToLower(), f.Identifier);
			}
		}
		// If the normal convention of naming the PK with the table name + Id, we may infer those
		foreach (var f in SchemaFields.Where(f => f.IsPrimary != "1" && f.ReferenceType == FieldReferenceType.NotAReference))
		{
			if (tempLookup.ContainsKey(f.FieldName.ToLower()))
			{
				f.Reference = tempLookup[f.FieldName.ToLower()];
				f.ReferenceType = FieldReferenceType.Inferred;
				count++;
			}
		}
		// Alternative Short Form convention where PK is always ID, but is prefixed with the singularized table name when referenced
		// e.g. table = MyElements pk = Id, fk = MyElementId
		foreach (var f in SchemaFields.Where(f => f.IsPrimary == "1" && f.FieldName.ToLower() == "id"))
		{
			var correctedKey = $"{f.TableName.ToLower()}id";
			var correctedReference = $"{f.SchemaName}.{f.TableName.ToLower()}.id";
			foreach (var g in SchemaFields.Where(f => f.IsPrimary != "1" && f.ReferenceType == FieldReferenceType.NotAReference))
			{
				if (tempLookup.ContainsKey(correctedKey))
				{
					g.Reference = correctedReference;
					g.ReferenceType = FieldReferenceType.Inferred;
					count++;
				}
			}
		}
		return count;
	}
	[Flags]
	public enum Navigability
	{
	    None = 0,
		InBound = 1,
		OutBound = 2,
		Both = 3
	}
	/////////////////////////////////////////////////////////////////////////////////////////////////////
	////
	public List<string> IdentifyReferences(string initialIdentifier, int maxDepth = 8, Navigability navigability = Navigability.Both, List<string>? result = null)
	{
		maxDepth--;
		if (result == null) result = new List<string>();
		if (maxDepth == 0)
		{
			return result;
		}
		// Inbound from FKs to this PK
		if ((navigability & Navigability.InBound) == Navigability.InBound)
		{
			var target =  SchemaFields.Where(f => f.Reference == initialIdentifier).DistinctBy(f => f.Identifier).Select(f => f.Identifier);
			foreach (var e in target)
			{
				if (!WasTraversed(e))
				{
					result.Add(e);
					var x = IdentifyReferences(e, maxDepth + 1, navigability, result).Where(e => !WasTraversed(e));
					result.AddRange(x);
				}
			}
			
		}
		// Outbound
		if ((navigability & Navigability.OutBound) == Navigability.OutBound)
		{
			var root = SchemaFields.Where(f => f.Identifier == initialIdentifier).First();
			if (!string.IsNullOrEmpty(root.Reference) && !WasTraversed(root.Reference))
			   result.Add(root.Reference);
			var target = SchemaFields
				.Where(f => f.SchemaName == root.SchemaName && f.TableName == root.TableName && !string.IsNullOrEmpty(f.Reference))
				.DistinctBy(f => f.Identifier)
				.Select(f => f.Identifier);
			foreach (var e in target)
			{
				if (!WasTraversed(e))
				{
					result.Add(e);
					var x = IdentifyReferences(e, maxDepth, navigability, result).Where(e => !WasTraversed(e));
					result.AddRange(x);
				}
			}
		}
		bool WasTraversed(string identifier)
		{
			return result.Contains(identifier);
		}
		return result;
	}
	public SchemaField this[int index]
	{
		get
		{
			if (SchemaFields.Length > index && index > -1)
				return SchemaFields[index];
			return new SchemaField();
		}
	}
	public SchemaField this[string  identifier]
	{
		get
		{
			return SchemaFields.FirstOrDefault(f => f.Identifier == identifier.ToLower()) ?? new SchemaField();
		}
	}
}
// public class ModelFilter
// {
// 	public string FieldName { get; set; } = "";
// 	public string FieldType { get; set; } = "";
// 	public string FieldValue { get; set; } = "";
// }
public class NestedData
{
	// this is the original Nest class
}
public class PresenterOptions
{
	// all the display options for css and script generation
}
// enum FieldDataTypes  //!  not currenly in use...
// {
// 	Bit,
// 	Date,
// 	Datetime,
// 	Decimal,
// 	Geography,
// 	Hierarchyid,
// 	Int,
// 	Money,
// 	Nchar,
// 	Numeric,
// 	Nvarchar,
// 	Smallint,
// 	Smallmoney,
// 	Time,
// 	Tinyint,
// 	Uniqueidentifier,
// 	Varbinary,
// 	Varchar,
// 	Xml
// }

In [ ]:
// |> some simple integration tests 
var modeler = new SqlSchemaModeler(_credentials);
var schemaNames = await modeler.GetSchemaNames();
// schemaNames.Display();
await modeler.InitializeAsync(schemaNames[1]);
//modeler.Diagnostic.Display();//
int x = modeler.Model.SchemaFields.Count();
await modeler.InitializeAsync(schemaNames[4]);
int y = modeler.Model.SchemaFields.Count();
await modeler.InitializeAsync(schemaNames[1], schemaNames[4]);
int z = modeler.Model.SchemaFields.Count();
Console.WriteLine($"{x} + {y} = {z}");
await modeler.InitializeAsync();
Console.WriteLine($"Total = {modeler.Model.SchemaFields.Count()}");
modeler.Diagnostic = "";
await modeler.InitializeAsync(schemaNames[0], schemaNames[1]);
//modeler.Model.SchemaFields.Where(f => f.Reference != null).Display();
//modeler._lookup.ContainsKey("humanresources.department.departmentid").Display();
//modeler.Model.SchemaFields.Where(f => f.ReferenceType.ToString() == "External").Display();
Console.WriteLine("expect sales.creditcard.creditcardid");
Console.WriteLine(modeler.Model.SchemaFields[99].Identifier);
await modeler.InitializeAsync(schemaNames[4]);
modeler.Model.SchemaFields.Select(f => f).DisplayTable();

In [ ]:
// |> verify schema overlay, refresh and prune work
var o1 = modeler.Model.SchemaFields[114];
o1.Reference.Display();
o1.Reference = "parameter.entrycustomerid";
await modeler.Overlay(o1);
var o2 = modeler.Model.SchemaFields[114];
o2.Reference.Display();
o1.IsExcluded = true;
modeler.Model.SchemaFields.Length.Display();// base
await modeler.Prune(true, false);
modeler.Model.SchemaFields.Length.Display();// should be the same
await modeler.Prune(false, true);
modeler.Model.SchemaFields.Length.Display();// should be one less
await modeler.Refresh();
modeler.Model.SchemaFields.Length.Display();// should be a whole lot more because not limited by schema anymore

In [ ]:
// |> some additional sanity checks
var ix = modeler.Model.SchemaFields.Any(f => f.ReferenceType == FieldReferenceType.External);
modeler["sales.creditcard.creditcardid"].ToString().Display();
modeler[519].ToString().Display();
modeler[-122].ToString().Display();
//modeler[ix].ToString().Display();
modeler.Model.SchemaFields.Length.Display();
Console.WriteLine(await modeler.ExcludebyPath("pe*.*.*"));
await modeler.Prune( false, true);
ix.Display();

In [ ]:
// |> add implied references and test
// Then create a routing table of sorts...
int count = await modeler.Model.InferReferences(true);
count.Display();
modeler.Model.SchemaFields.Where(f => f.IsInferredReference).Select(f => f.ToString()).DisplayTable();

In [ ]:
// |> having inferred refs, serialize and deserialize the model
int n = modeler.FieldCount;
FileData.ToFile(modeler.ToJson(), "../../_sand-box/.data/Model00.jmodel");
Console.WriteLine($"{modeler.FieldCount} fields should be {n}.");
modeler.Model = new SchemaModel();
Console.WriteLine($"{modeler.FieldCount} fields should be zero.");
modeler.FromJson(FileData.FromFile("../../_sand-box/.data/Model00.jmodel"));
Console.WriteLine($"{modeler.FieldCount} fields should be {n}.");



_The recursive path had some issues and needed some debugging (in LinqPad)_ 

In [ ]:
// |> demo the refences idenfier
var startIdentifier = "sales.customer.customerid";
var x = modeler.Model.IdentifyReferences(startIdentifier, 1, SchemaModel.Navigability.OutBound);
Console.WriteLine($"Depth = 0, expect 0 items: {x.Count()}");
x = modeler.Model.IdentifyReferences(startIdentifier, 8, SchemaModel.Navigability.Both);
Console.WriteLine($"Depth = 0, expect 0 items: {x.Count()}");
foreach (var s in x.OrderBy(x => x))
  Console.WriteLine(modeler[s]);


## Continuation

The SchemaModeler is now working okay, and we have the ability to capture inferred and defined paths from any starting identifier.

### Reminder of the flow:

<pre>
SqlSchemaModeler --< SchemaModel >--> SqlModelQuery --< JsonQueryResult >--> NestedDataModeler --< NestedData >--> Publisher --< Page >-->
        * mainly good     
                                           * should accept the new form of links from the model
                                           * will inject the indexes into the field names
                                                                                   * will intercept the indexes and interpret during parsing to the nested data
                                                                                   * can remove or inject nested data as needed
                                                                                                                        *  should not care too much
                                                                                                                        *  could use plenty of css love
</pre>        
Next step is to make the new SqlModelQuery class (from the old one) and the generic NestedData class(from the old one too) - we are just separating some concerns to get the non-agnostic pieces out of the nest class.

The old Nest class had a mixture of Html/Json and css that does not belong there: the actual class is extremely simple, and simply generates a NestedData hierarchy from a JsonObject or a Json string.

In [ ]:
// |> Json helper class formerly embedded in the Nest class
public class JsonHelpers
{
    /// <summary>
    /// Returns a json serialization of the supplied object.
    /// </summary>
    public static string AsJson(object obj)
    {
        return JsonSerializer.Serialize(obj);
    }
    /// <summary>
    /// Returns a json string formatted with indentation and line breaks.
    /// </summary>
    public static string Prettify(string json) //!
    {
        JsonSerializerOptions options = new JsonSerializerOptions
        {
            WriteIndented = true,
            MaxDepth = 16,
            ReferenceHandler = ReferenceHandler.Preserve

        };
        JsonDocument doc = JsonDocument.Parse(json);
        using (MemoryStream stream = new MemoryStream())
        {
            using (
                Utf8JsonWriter writer = new Utf8JsonWriter(
                    stream,
                    new JsonWriterOptions { Indented = true }
                )
            )
            {
                doc.WriteTo(writer);
            }
            return Encoding.UTF8.GetString(stream.ToArray());
        }
    }
}

In [ ]:
// |> Nested Data class derived from former Nest class

/* This class looks like it could be generic, with a known Vlue type maybe. So far this has not been necessary, as the focus has been on json-derializable things, which are okay with text values.
One opossibility would be to add a Raw<T> memeber if needed. */

#nullable enable

public class NestedData
{
    public NestedData? Parent { get; set; }

    /// <summary/May be used to track the instance number of a series of NestedDatas. For example, siblings.</summary>
    public int Instance { get; set; } = 1;
    public enum NestType
    {
        Unknown = 0,
        Set,       // Json Object, heterogeneous array
        List,     // Json Array,  homogeneous array
        SetList, // Array of Objects
        Item,   // Value 
    }

    public int Depth { get; set; } = 0;
    public IEnumerable<NestedData> Children { get; set; } = new NestedData[0];
    public string? Name { get; set; } = "";
    public string? Value { get; set; } = "";
    public NestType Nesting { get; set; } = NestType.Unknown;

    public string TagId // 
    {
        get
        {
            // To support deep nesting, it is necessary to build a chain of the instanceids.
            // This will allow individual identities within the fully expanded tree,
            // which is essential to identify targets for Javascript actions later. 
            string x = $"-{Instance}";
            if (Parent == null)
                return $"{Name}{x}";
            NestedData? p = Parent;
            do
            {
                x = $"-{p.Instance}{x}";
                p = p.Parent;
            } while (p != null);
            return $"{Parent.Name}{x}";
        }
    }

    ///  construct from Json string

    public NestedData(string name, string json)
        : this(name, JsonNode.Parse(json)) { }

    /// construct from JsonNode
    public NestedData(
        string name,
        System.Text.Json.Nodes.JsonNode? node,
        int depth = 0,
        int instance = 1
    )
    {
        Depth = depth;
        /* Need language version 9.0)
       /* Nesting = node switch
        {
            JsonObject => NestType.Set,
            JsonArray => NestType.List,
            JsonValue => NestType.Item,
            _ => NestType.Item
        }; */
        if (node is JsonObject) { Nesting = NestType.Set; }
        else if (node is JsonArray) { Nesting = NestType.List; }
        else /*if (node is JsonValue)*/ { Nesting = NestType.Item; }
        Instance = instance;
        Name = name;
        Children = new List<NestedData>();
        if (node is JsonObject obj)
        {
            int childInstance = 1;
            foreach (var kv in obj)
            {
                //if (kv.Value != null)
                //{
                    Children = Children.Append(
                        new NestedData(kv.Key, kv.Value, depth + 1, childInstance++) { Parent = this }
                    );
                //}
            }
        }


        else if (node is JsonArray arr)
        {
            int arrayInstance = 1;
            foreach (var jnq in arr)
            {
                if (jnq is JsonObject)
                {
                    Children = Children.Append(
                        new NestedData($"{Name}", (JsonObject)jnq, Depth + 1, arrayInstance++)
                        {
                            Parent = this
                        }
                    );
                }
                else if (jnq != null)
                {
                    Children = Children.Append(
                        new NestedData(
                            $"{Name}({arrayInstance} of {arr.AsArray().Count()})",
                            jnq,
                            Depth + 1,
                            arrayInstance++
                        )
                    );
                }
            }
        }
        else if (node is JsonValue val)
        {
            Value = val.ToString();
        }
        if (Nesting == NestType.List)
        {
            if (Children.All(n => n.Nesting == NestType.Set))
            {
                Nesting = NestType.SetList;
            }
        }
    }
    public override string ToString() =>
        $"{Name}-{Instance}{(Parent == null ? "" : $"-{Parent.Instance}")}";

    public string AllElements(bool WithLineBreaks = false)
    {
        string between = WithLineBreaks ? "\n" : "";
        StringBuilder sb = new StringBuilder();
        foreach (string s in Elements())
        {
            sb.Append($"{s}{between}");
        }
        return sb.ToString();
    }
    public IEnumerable<string> Elements()
    {
        yield return RenderText();
        foreach (var child in Children)
        {
            foreach (var x in child.Elements())
            {
                yield return x;
            }
        }
        yield break;

        string RenderText()
        {
            switch (Nesting)
            {
                case NestType.Set:
                    return $"<h{Depth}> Set {Name} (has {Children.Count()} Elements) </h{Depth}>";
                case NestType.List:
                    return $"<h{Depth}> List {Name} (has {Children.Count()} items) </h{Depth}>";
                case NestType.Item:
                default:
                    return $"<h{Depth}> {Name} = {Value} </h{Depth}>";
                case NestType.SetList:
                    return $"<h{Depth}> SetList {Name} (has {Children.Count()} sets of {Children.First().Children.Count()} elements) </h{Depth}>";
            }
        }
    }
}

In [ ]:
// |> small throw-away object definition for testing the nested data class

public const string Bands = "{ \"Bands\": ["
+"{ \"Name\": \"Beatles\",\"Members\": ["
+"{ \"BandMember\": \"John\","
+"\"Instruments\": [\"Harmonica\", \"Guitar\", \"Vocals\"]},"
+"{ \"BandMember\": \"Paul\","
+"\"Instruments\": [\"Bass\", \"Guitar\", \"Keyboards\", \"Vocals\"]},"
+"{ \"BandMember\": \"Ringo\", \"Instruments\": [\"Drums\", \"Vocals\"]},"
+"{ \"BandMember\": \"George\",\"Instruments\": [\"Harmonica\", \"Guitar\", \"Sitar\", \"Vocals\"]}"
+"]},{\"Name\": \"Basemen\",\"Members\": ["
+"{ \"BandMember\": \"Miles\", \"Instruments\": [\"Drums\", \"Vocals\"]},"
+"{ \"BandMember\": \"Alan\", \"Instruments\": [\"Bass\", \"Keyboards\", \"Vocals\"]},"
+"{ \"BandMember\": \"Tim\", \"Instruments\": [\"Keyboards\", \"Vocals\"]},"
+"{ \"BandMember\": \"Nick\", \"Instruments\": [\"Harmonica\", \"Guitar\", \"Keyboards\", \"Bass\", \"Vocals\"]}"
+"]}]}";

JsonHelpers.Prettify(Bands).Display();


In [ ]:
// |> demo NestedData from the above

var data = new NestedData("Bands", Bands);
data.AllElements(true).Display();

In [ ]:
// |> Copied the output above into this to view as html -->
#!html
<h0> Set Bands (has 1 Elements) </h0>
<h1> SetList Bands (has 2 sets of 2 elements) </h1>
<h2> Set Bands (has 2 Elements) </h2>
<h3> Name = Beatles </h3>
<h3> SetList Members (has 4 sets of 2 elements) </h3>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = John </h5>
<h5> List Instruments (has 3 items) </h5>
<h6> Instruments(1 of 3) = Harmonica </h6>
<h6> Instruments(2 of 3) = Guitar </h6>
<h6> Instruments(3 of 3) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Paul </h5>
<h5> List Instruments (has 4 items) </h5>
<h6> Instruments(1 of 4) = Bass </h6>
<h6> Instruments(2 of 4) = Guitar </h6>
<h6> Instruments(3 of 4) = Keyboards </h6>
<h6> Instruments(4 of 4) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Ringo </h5>
<h5> List Instruments (has 2 items) </h5>
<h6> Instruments(1 of 2) = Drums </h6>
<h6> Instruments(2 of 2) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = George </h5>
<h5> List Instruments (has 4 items) </h5>
<h6> Instruments(1 of 4) = Harmonica </h6>
<h6> Instruments(2 of 4) = Guitar </h6>
<h6> Instruments(3 of 4) = Sitar </h6>
<h6> Instruments(4 of 4) = Vocals </h6>
<h2> Set Bands (has 2 Elements) </h2>
<h3> Name = Basemen </h3>
<h3> SetList Members (has 4 sets of 2 elements) </h3>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Miles </h5>
<h5> List Instruments (has 2 items) </h5>
<h6> Instruments(1 of 2) = Drums </h6>
<h6> Instruments(2 of 2) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Alan </h5>
<h5> List Instruments (has 3 items) </h5>
<h6> Instruments(1 of 3) = Bass </h6>
<h6> Instruments(2 of 3) = Keyboards </h6>
<h6> Instruments(3 of 3) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Tim </h5>
<h5> List Instruments (has 2 items) </h5>
<h6> Instruments(1 of 2) = Keyboards </h6>
<h6> Instruments(2 of 2) = Vocals </h6>
<h4> Set Members (has 2 Elements) </h4>
<h5> BandMember = Nick </h5>
<h5> List Instruments (has 5 items) </h5>
<h6> Instruments(1 of 5) = Harmonica </h6>
<h6> Instruments(2 of 5) = Guitar </h6>
<h6> Instruments(3 of 5) = Keyboards </h6>
<h6> Instruments(4 of 5) = Bass </h6>
<h6> Instruments(5 of 5) = Vocals </h6>


## SqlModelQuery class

Refactored from code originally contained in the JsonSqlClient.

This version differs in that it needs to

- use a model as its starting point instead of reflecting the database
- output encoded indexes into the model as column names
- accept and inject filters or limit the scope in other means
- the JsonQueryResult will be modeled into a NestedData object by the NestedDataModeler.
The SqlModelQuery is an implementation specific to Sql Server, whereas the NestedDataModeler should be able to work with ANY SChemaModel and JsonData as input, which is why the classes are separated.


In [ ]:
// |> SqlModelQuery
/* This class will derive from the IModelQuery interface to allow substitution. The NestedDataModeler is a dependency of this class, as it is used to convert the intermediary raw json data to NestedData.
For privacy concerns, the raw json should not be exposed and the NestedDataModeler is therefore instantiated internally to protect the data until it is safely exposed as NestedData. */

// The link class is a dependency of the SqlModelQuery, used to traverse the mnodel.
public class Link
{
    public string Outer { get; set; }
    public string Inner { get; set; }

    public bool IsReversed { get; private set; } = false;

    /// <summary>
    /// Defines a dependency between two tables.
    /// Each of the two main elements is a string consisting of the <schemaName>.<tableName>.<columnName>.
    /// this information will be used to join the subqueries.
    /// The optional limit (defaults to 10) limits the select statement to the most recent identity values.
    /// </summary>
    /// <param name="outer">The schema.table.column of the primary key</param>
    /// <param name="inner">The schema.table.column of the foreign key</param>
    /// <param name="limit">The limit to apply to the top select (if applicable to the query type)</param>
    public Link(string outer, string inner, int limit = 10)
    {
        Inner = inner;
        Outer = outer;
        Limit = limit;
    }

    public void Flip()
    {
        string temp = Outer;
        Outer = Inner;
        Inner = temp;
        IsReversed = !IsReversed;
    }

    public override string ToString() => $" {Outer} <== {Inner}" ;

    public int Limit { get; set; }
    public string InnerTableName =>
        Inner.Substring(InnerDots[0] + 1, InnerDots[1] - InnerDots[0] - 1);
    public string OuterTableName =>
        Outer.Substring(OuterDots[0] + 1, OuterDots[1] - OuterDots[0] - 1);
    public string InnerTableFullName => Inner.Substring(0, InnerDots[1]);
    public string OuterTableFullName => Outer.Substring(0, OuterDots[1]);
    public string InnerKeyFullName => Inner;
    public string OuterKeyFullName => Outer;
    public string InnerColumnName => Inner.Substring(InnerDots[1] + 1);
    public string OuterColumnName => Outer.Substring(OuterDots[1] + 1);
    public string MatchPhrase => $" {Inner} = {Outer}";
    // used for string parsing
    private int[] InnerDots => new int[] { Inner.IndexOf('.'), Inner.LastIndexOf('.') };
    private int[] OuterDots => new int[] { Outer.IndexOf('.'), Outer.LastIndexOf('.') };

}


#nullable enable

public class SqlModelQuery // : IModelQuery
{
    private SchemaModel _model;
    private JsonSqlClient _client;
    public string Diagnostic  {get; set; } = "";

    // Inject both the model and the client, so that scope is limited to a single client and model, but potentially multiple query executins.
    public SqlModelQuery(string connectionString, SchemaModel model)
    {  _model = model;
       _client = new JsonSqlClient(connectionString);
    }

    // this was the orginal version using links:
    // public string[] GenerateNestedQuery(params Link[] links)
    // {
    //     string[] result = new String[2] { "", "" };
    //     ProcessMultiLinkQuery(ref result, links.ToList(), null);
    //     return result;
    // }
    // this is what we will use temporararily: later this will roll into the NestedDataModeler
    // the first string in the result is the query, the second is a descriptive piece - this might change.


    // the filter should be a string with two parts.  The first is the identifier, the second is a string representing the operation and value.
    private (string Identifier, string Condition) ParseFilter(string filter)
    {
        // assume the only operators at this stage is =
        // later we might add:  like, between, <, > 
        string safeFilter = filter.Trim().ToLower();
        int index = safeFilter.IndexOfAny("<=>".ToCharArray());
        if (index < 0)
        {  // not an equals, less than or greater than, so we expect a space separator.
           index = safeFilter.IndexOf(' ');
           if (index <0) 
              throw new DataException("Filter should consist of a fully qualified Schema.Table.Column followed by the condition, with a space delimiter unless ==, < or >");
        }
        string identifier = safeFilter.Substring(0, index).Trim(); // case is made lower.
        string condition = filter.Trim().Substring(index); // case is preserved.
        // Sql injection could occur if an unmatched single quote is inside the condition
  
        
        return (identifier, condition);
    }
    private string[] GetColumnNames(string identifier)
    {   
        return
        _model.SchemaFields
           .Where(f => f.TableName.ToLower() == _model[identifier].TableName.ToLower())
           .Where(f => f.CanShow)
           .Select(f => f.FieldName)
           .ToArray();
    }
    // Initially: We simply have the model, and the max Depth, followed by one or more filters
    // The first filter (in the form schema.table.field=value) triggers the start point 
    public string[] ExecuteQuery(int maxDepth, params string[] filterConditions)
    {
        
        string[] result = new String[2] { "", "" }; // this is used for recursion. The start and end strings are passed, ultimately the istart string contains everything.
        (string Identifier, string Condition) [] filters = filterConditions.Select(f => ParseFilter(f)).ToArray();
        var sequence = _model.IdentifyReferences(filters[0].Identifier, maxDepth, SchemaModel.Navigability.Both);
        // this is a sequence of identifiers. Each represents a link between 2 tables, and the reference is explicitly identified in the SchemaFields entry.
        var links = sequence.Select(s => new Link(s, _model[s].Reference, maxDepth)).ToList();
        ProcessModelQuery(ref result, links, null);
        return result;
    }

    // this is recursive core of the processor.
    private void ProcessModelQuery(
        ref string[] result,
        List<Link> links,  /// this is the identifiers of the primary key field of the link
        Link? current = null
    )
    {
Diagnostic = $"133 {current}";
        if (current == null)
        {
            //! use the first link to start the series
            current = links[0];
            result[1] = $"Root: {current}";
            AddAsRoot(ref result, current);
            links.Remove(current);
Diagnostic = $"141 {current}";
            ProcessModelQuery(ref result, links, current);
        }
        else
        {
            var candidate = links.FirstOrDefault(l => IsNest(current, ref l));
Diagnostic = $"147 : {candidate}";            
            if (candidate != null)
            {
                result[1] += $"\nNest: {candidate}";
                AddAsNest(ref result, candidate);
                links.Remove(candidate);
Diagnostic = "153";
                ProcessModelQuery(ref result, links, candidate);
                //output[0] += output[1];
            }
            // we should exhaust all peers before updating the current
            while (links.Any(l => IsPeer(current, ref l)))
            {
                Link link = links.First(l => IsPeer(current, ref l));
                result[1] += $"\nPeer: {link}";
                AddAsPeer(ref result, link);
                links.Remove(link);
Diagnostic = "164";                
                ProcessModelQuery(ref result, links, link);
            }
            current = links.FirstOrDefault(
                l => IsNest(current, ref l) || IsPeer(current, ref l)
            );
        }

        // local methods
        // Remember, a LINK involves 2 tables! an Inner and an Outer. A Nest or Peer defines how two Links interact with a common table
        // A Nest occurs when the inner of one is the outer of the other
        // A Peer relationship occurs when two lks share a common outer.
        bool IsNest(Link parent, ref Link child)
        {
            if (child.OuterTableFullName.ToUpper() == parent.InnerTableFullName.ToUpper())
            {
                return true;
            }
            if (child.InnerTableFullName.ToUpper() == parent.InnerTableFullName.ToUpper())
            {
                child.Flip();
                return true;
            }
            return false;
        }
        bool IsPeer(Link parent, ref Link child)
        {
            if (parent.OuterTableFullName.ToUpper() == child.InnerTableFullName.ToUpper())
            {
                return true;
            }
            if (parent.OuterTableFullName.ToUpper() == child.OuterTableFullName.ToUpper())
            {
                child.Flip();
                return true;
            }
            return false;
        }
        void AddAsRoot(ref string[] output, Link link)
        {
            // creates a new nest from two tables
            string innerFields = string.Join(",", GetColumnNames(link.InnerTableFullName));
            string outerFields = string.Join(",", GetColumnNames(link.OuterTableFullName));
            output[0] =
                $"select top {link.Limit} {outerFields}, ( select {innerFields} from {link.InnerTableFullName} where {link.MatchPhrase} for json path, include_null_values ) as"
                + $" {link.InnerTableName} from {link.OuterTableFullName} order by {link.OuterKeyFullName} desc for json path, root('{link.OuterTableName}'), include_null_values";
        }
        void AddAsPeer(ref string[] output, Link link)
        {
            // inserts a peer with a common outer table which already exists.
            // the insertion occurs immediately before the "from {OuterTableFullName} " so is easy to find.
            int index = output[0].IndexOf($" from {link.OuterTableFullName} ");
            if (index < 0)
            {
                link.Flip();
                index = output[0].IndexOf($" from {link.OuterTableFullName} ");
            }
            if (index < 0)
            {
                output[1] += $"\nUnable to find peer for link {link}";
                return;
            }
            string temp = output[0].Substring(index);
            string innerFields = string.Join(",", GetColumnNames(link.InnerTableFullName));
            output[0] = output[0].Substring(0, index - 1);
            output[0] +=
                $", (  select top({link.Limit}) {innerFields} from {link.InnerTableFullName} where {link.MatchPhrase} for json path, include_null_values ) as"
                + $" {link.InnerTableName}"
                + temp;
        }
        void AddAsNest(ref string[] output, Link link)
        {
            // inserts a nest within an existing outer table
            int index = output[0].IndexOf($" from {link.OuterTableFullName} where ");
            if (index < 0)
            {
                throw new Exception($"{link.OuterTableFullName} not found in existing query.");
            }
            string temp = output[0].Substring(index);
            string innerFields = string.Join(",", GetColumnNames(link.InnerTableFullName));
            output[0] = output[0].Substring(0, index + 1);
            output[0] +=
                $", (  select top({link.Limit}) {innerFields} from {link.InnerTableFullName} where {link.MatchPhrase} for json path, include_null_values ) as"
                + $" {link.InnerTableName}"
                + temp;
        }
    }
}


In [ ]:
// |> Test the first phase of query execution
var x = modeler.Model.IdentifyReferences("sales.creditcard.creditcardid");
x.Display();
x.Count().Display();

var query = new SqlModelQuery(_credentials, modeler.Model);
var result = query.ExecuteQuery( 8, "sales.creditcard.creditcardid=127");
result.Display();

In [ ]:
// |> Show diagnostics from the execution
query.Diagnostic.Display();